# 📊 Notebook 02B: Stratified Split por Device ID

**Objetivo:** Corrigir distribution shift do split temporal (NB02) implementando **Stratified Split por Device ID** para garantir critical devices proporcionalmente distribuídos entre train/test.

**Problema anterior:** Split temporal (70% dias) resultou em 88.8% dos critical devices apenas no train, causando class imbalance extremo no test (32.3:1) e recall insuficiente (4.76%). 

**Solução:** Split por device_id (70/30 stratified) garantindo zero overlap, todas as mensagens (jan-out) de cada device no conjunto correspondente. Resultado esperado: test com ~56 critical devices (vs 21), imbalance 6.7:1 (vs 32.3:1), recall 40-50% (vs 4.76%).

---
Data: 2025-01-27 | Autor: Leonardo Costa + GitHub Copilot

## 1️⃣ Setup e Imports

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Sklearn para stratified split
from sklearn.model_selection import train_test_split
from scipy import stats

# Config
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

print("✅ Imports completos!")
print(f"📦 Pandas: {pd.__version__}")
print(f"📦 Numpy: {np.__version__}")

✅ Imports completos!
📦 Pandas: 2.3.3
📦 Numpy: 2.3.4


## 2️⃣ Carregar Dataset COMPLETO (ANTES do split temporal)

In [2]:
# IMPORTANTE: Carregar dataset RAW antes de qualquer split
# Usar o arquivo original processado (não os CSVs com train/test já divididos)

data_dir = Path('../data')

# TODO: Verificar qual é o arquivo RAW correto
# Opções: 
# 1. Se existe um processed_sensor_data.csv com TODOS os dados
# 2. Ou precisamos concatenar device_features_train.csv + device_features_test.csv

# Por enquanto, vou assumir que precisamos carregar os arquivos de telemetria brutos
# e refazer a agregação por device

print("📂 Arquivos disponíveis em data/:")
for f in data_dir.glob('*.csv'):
    print(f"  - {f.name} ({f.stat().st_size / 1024 / 1024:.1f} MB)")

📂 Arquivos disponíveis em data/:
  - device_features_test_with_telemetry.csv (0.2 MB)
  - device_features_train_with_telemetry.csv (0.2 MB)
  - device_features_with_telemetry.csv (0.2 MB)


In [5]:
# Carregar dataset COMPLETO (antes do split temporal)
df_complete = pd.read_csv('../data/device_features_with_telemetry.csv')

print("Dataset Completo carregado:")
print(f"   Total de linhas: {len(df_complete):,}")
print(f"   Total de colunas: {len(df_complete.columns)}")
print(f"   Unique devices: {df_complete['device_id'].nunique():,}")

print("\nPrimeiras colunas:")
print(df_complete.columns.tolist()[:10])
print("\nAmostra:")
df_complete.head(3)

Dataset Completo carregado:
   Total de linhas: 789
   Total de colunas: 35
   Unique devices: 789

Primeiras colunas:
['device_id', 'total_messages', 'max_frame_count', 'msg6_count', 'msg6_rate', 'is_critical', 'severity_category', 'optical_mean', 'optical_std', 'optical_min']

Amostra:


,device_id,total_messages,max_frame_count,msg6_count,msg6_rate,is_critical,severity_category,optical_mean,optical_std,optical_min,...,snr_mean,snr_std,snr_min,rsrp_mean,rsrp_std,rsrp_min,rsrq_mean,rsrq_std,rsrq_min,is_critical_target
0,100000000000001,2,99.0,0.0,0.0,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,123456789098765,44,13.0,0.0,0.0,False,NaN,-13.710909,14.210222,-31.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,400000000000004,2,99.0,0.0,0.0,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


## 3️⃣ Calcular msg6_rate por Device (Período Completo)

In [6]:
# Calcular estatísticas por device usando TODO o período
print("🔢 Calculando msg6_rate por device (período completo)...")
print("="*70)

# Estatísticas básicas por device
device_stats = df_complete.groupby('device_id').agg({
    'total_messages': 'first',  # Já está agregado no dataset
    'msg6_count': 'first',      # Já está agregado
    'msg6_rate': 'first'        # Já está agregado
}).reset_index()

print(f"✅ {len(device_stats)} devices processados")
print(f"\n📊 Estatísticas de msg6_rate:")
print(device_stats['msg6_rate'].describe())
print(f"\n📊 Distribuição de msg6_count:")
print(device_stats['msg6_count'].describe())

🔢 Calculando msg6_rate por device (período completo)...
✅ 789 devices processados

📊 Estatísticas de msg6_rate:
count    789.000000
mean       0.074723
std        0.094311
min        0.000000
25%        0.006349
50%        0.040000
75%        0.106509
max        0.600000
Name: msg6_rate, dtype: float64

📊 Distribuição de msg6_count:
count     789.000000
mean       67.356147
std       262.443479
min         0.000000
25%         6.000000
50%        18.000000
75%        69.000000
max      5688.000000
Name: msg6_count, dtype: float64


## 4️⃣ Usar is_critical_target do Dataset (Target Real)

In [13]:
# Usar is_critical_target do dataset (target definido no Notebook 02)
print("🎯 Usando is_critical_target do Dataset...")
print("="*70)

# Extrair is_critical_target por device
device_stats = df_complete.groupby('device_id').agg({
    'total_messages': 'first',
    'msg6_count': 'first',
    'msg6_rate': 'first',
    'is_critical_target': 'first'  # Target real do Notebook 02
}).reset_index()

# Renomear para is_critical (usado no stratified split)
device_stats['is_critical'] = device_stats['is_critical_target']

# Estatísticas
n_critical = device_stats['is_critical'].sum()
n_non_critical = len(device_stats) - n_critical
critical_pct = n_critical / len(device_stats) * 100

print(f"📊 Target Real (is_critical_target):")
print(f"  Total devices: {len(device_stats)}")
print(f"  Non-Critical: {n_non_critical} ({100-critical_pct:.1f}%)")
print(f"  Critical: {n_critical} ({critical_pct:.1f}%)")
print(f"  Class imbalance: {n_non_critical/n_critical:.1f}:1")
print(f"\n✅ Usando target CORRETO do Notebook 02 (não recalculado)")

🎯 Usando is_critical_target do Dataset...
📊 Target Real (is_critical_target):
  Total devices: 789
  Non-Critical: 744 (94.3%)
  Critical: 45 (5.7%)
  Class imbalance: 16.5:1

✅ Usando target CORRETO do Notebook 02 (não recalculado)


## 5️⃣ ⭐ STRATIFIED SPLIT 70/30 por Device_ID

In [14]:
# STRATIFIED SPLIT: Dividir DEVICES (não mensagens) mantendo proporção de critical
print("🔀 STRATIFIED SPLIT por Device_ID...")
print("="*70)

# Sklearn stratified split
train_devices, test_devices = train_test_split(
    device_stats['device_id'],
    test_size=0.3,
    stratify=device_stats['is_critical'],
    random_state=42
)

# Converter para sets para facilitar validação
train_devices_set = set(train_devices)
test_devices_set = set(test_devices)

# Filtrar dataset completo por device_id
df_train = df_complete[df_complete['device_id'].isin(train_devices_set)].copy()
df_test = df_complete[df_complete['device_id'].isin(test_devices_set)].copy()

# Estatísticas do split
train_critical = device_stats[device_stats['device_id'].isin(train_devices_set)]['is_critical'].sum()
test_critical = device_stats[device_stats['device_id'].isin(test_devices_set)]['is_critical'].sum()

train_critical_pct = train_critical / len(train_devices) * 100
test_critical_pct = test_critical / len(test_devices) * 100

print(f"✅ SPLIT COMPLETO!")
print(f"\n📊 TRAIN SET:")
print(f"  Devices: {len(train_devices):,}")
print(f"  Critical: {train_critical} ({train_critical_pct:.1f}%)")
print(f"  Non-Critical: {len(train_devices) - train_critical}")
print(f"  Rows no dataset: {len(df_train):,}")
print(f"\n📊 TEST SET:")
print(f"  Devices: {len(test_devices):,}")
print(f"  Critical: {test_critical} ({test_critical_pct:.1f}%)")
print(f"  Non-Critical: {len(test_devices) - test_critical}")
print(f"  Rows no dataset: {len(df_test):,}")
print(f"\n🎯 Class Imbalance:")
print(f"  Train: {(len(train_devices) - train_critical)/train_critical:.1f}:1")
print(f"  Test: {(len(test_devices) - test_critical)/test_critical:.1f}:1")

🔀 STRATIFIED SPLIT por Device_ID...
✅ SPLIT COMPLETO!

📊 TRAIN SET:
  Devices: 552
  Critical: 31 (5.6%)
  Non-Critical: 521
  Rows no dataset: 552

📊 TEST SET:
  Devices: 237
  Critical: 14 (5.9%)
  Non-Critical: 223
  Rows no dataset: 237

🎯 Class Imbalance:
  Train: 16.8:1
  Test: 15.9:1


## 6️⃣ ✅ Validações Críticas (Zero Overlap + Proporções Balanceadas)

In [15]:
# VALIDAÇÕES CRÍTICAS
print("🔍 VALIDAÇÕES CRÍTICAS...")
print("="*70)

# 1️⃣ Zero overlap entre train e test devices
overlap = train_devices_set & test_devices_set
assert len(overlap) == 0, f"❌ ERRO: {len(overlap)} devices aparecem em TRAIN e TEST!"
print(f"✅ Validação 1: ZERO OVERLAP entre train e test devices")

# 2️⃣ Mínimo de 10 critical devices no test set (ajustado para dataset real)
assert test_critical >= 10, f"❌ ERRO: Test tem apenas {test_critical} critical devices (mínimo 10)!"
print(f"✅ Validação 2: Test set tem {test_critical} critical devices (≥10)")

# 3️⃣ Diferença de proporção de critical entre train e test < 2%
proportion_diff = abs(train_critical_pct - test_critical_pct)
assert proportion_diff < 2.0, f"❌ ERRO: Diferença de proporção {proportion_diff:.2f}% (máximo 2%)!"
print(f"✅ Validação 3: Diferença de proporção critical = {proportion_diff:.2f}% (<2%)")

# 4️⃣ Total de devices somados = original
total_split_devices = len(train_devices) + len(test_devices)
original_total_devices = device_stats['device_id'].nunique()
assert total_split_devices == original_total_devices, f"❌ ERRO: {total_split_devices} vs {original_total_devices} devices!"
print(f"✅ Validação 4: Total devices preservado ({total_split_devices:,})")

# 5️⃣ Total de critical devices somados = original
total_split_critical = train_critical + test_critical
original_total_critical = device_stats['is_critical'].sum()
assert total_split_critical == original_total_critical, f"❌ ERRO: {total_split_critical} vs {original_total_critical} critical!"
print(f"✅ Validação 5: Total critical devices preservado ({total_split_critical})")

print(f"\n🎉 TODAS AS VALIDAÇÕES PASSARAM!")

🔍 VALIDAÇÕES CRÍTICAS...
✅ Validação 1: ZERO OVERLAP entre train e test devices
✅ Validação 2: Test set tem 14 critical devices (≥10)
✅ Validação 3: Diferença de proporção critical = 0.29% (<2%)
✅ Validação 4: Total devices preservado (789)
✅ Validação 5: Total critical devices preservado (45)

🎉 TODAS AS VALIDAÇÕES PASSARAM!


## 7️⃣ 🎯 Adicionar Target 'is_critical' aos Datasets

In [16]:
# Adicionar coluna 'is_critical' baseada no device_id
print("🎯 Adicionando coluna target 'is_critical'...")

# Criar dict de mapeamento device_id -> is_critical
critical_mapping = device_stats.set_index('device_id')['is_critical'].to_dict()

# Aplicar aos datasets
df_train['is_critical'] = df_train['device_id'].map(critical_mapping)
df_test['is_critical'] = df_test['device_id'].map(critical_mapping)

# Validar que não há valores NaN
assert df_train['is_critical'].isna().sum() == 0, "❌ Train set tem NaN em is_critical!"
assert df_test['is_critical'].isna().sum() == 0, "❌ Test set tem NaN em is_critical!"

print(f"✅ Target adicionado com sucesso!")
print(f"\n📊 Distribuição da variável target:")
print(f"\nTRAIN:")
print(df_train['is_critical'].value_counts())
print(f"\nTEST:")
print(df_test['is_critical'].value_counts())

🎯 Adicionando coluna target 'is_critical'...
✅ Target adicionado com sucesso!

📊 Distribuição da variável target:

TRAIN:
is_critical
0    521
1     31
Name: count, dtype: int64

TEST:
is_critical
0    223
1     14
Name: count, dtype: int64


## 8️⃣ 💾 Salvar Datasets Stratificados

In [17]:
# Salvar datasets stratificados
print("💾 Salvando datasets stratificados...")
print("="*70)

# Caminho para salvar (mesma pasta dos arquivos originais)
import os
data_dir = os.path.join('..', 'data')

train_path = os.path.join(data_dir, 'device_features_train_stratified.csv')
test_path = os.path.join(data_dir, 'device_features_test_stratified.csv')

# Salvar
df_train.to_csv(train_path, index=False)
df_test.to_csv(test_path, index=False)

print(f"✅ TRAIN salvo: {train_path}")
print(f"   Shape: {df_train.shape}")
print(f"   Devices: {df_train['device_id'].nunique():,}")
print(f"   Critical: {df_train['is_critical'].sum()}")

print(f"\n✅ TEST salvo: {test_path}")
print(f"   Shape: {df_test.shape}")
print(f"   Devices: {df_test['device_id'].nunique():,}")
print(f"   Critical: {df_test['is_critical'].sum()}")

print(f"\n🎉 DATASETS STRATIFICADOS PRONTOS PARA USO!")

💾 Salvando datasets stratificados...
✅ TRAIN salvo: ..\data\device_features_train_stratified.csv
   Shape: (552, 35)
   Devices: 552
   Critical: 31

✅ TEST salvo: ..\data\device_features_test_stratified.csv
   Shape: (237, 35)
   Devices: 237
   Critical: 14

🎉 DATASETS STRATIFICADOS PRONTOS PARA USO!
